In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
import json

import numpy as np
import pandas as pd
import quantstats as qs

from pathlib import Path

from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt

from api_endpoints import url_account, url_positions
from api_methods import get_login_data, download_positions

In [ ]:
qs.extend_pandas()

plt.style.use('ggplot')
%matplotlib inline

# Data

In [ ]:
user_data = get_login_data()

In [ ]:
date_start = '20191204'
date_end   = '20191215'

In [ ]:
path = Path(r'')

Descargamos el excel con la composición del portfolio de cada día laborable:

In [ ]:
calendar = pd.date_range(start = date_start, end = date_end, freq = 'B')

In [ ]:
download_positions(calendar          = calendar, 
                   path              = path, 
                   data              = user_data,
                   filename_template = 'pos_%Y%m%d')

## Parse the Excel files and clean output

Una vez descargados los excel con el portfolio de cada dia, los cargamos en un DataFrame:

In [ ]:
columnas = ['Fecha','Producto','Symbol/ISIN','Cantidad','Precio de ','Valor local','Valor en EUR']

df = pd.DataFrame(columns=columnas)

for file in path.glob('pos*.xls'):
    
    # Read file
    df_day = pd.read_excel(file)
    
    # Get positions date
    df_day['Fecha'] = file.stem.split('_')[-1]
    
    # Append to dataframe
    df = df.append(df_day, sort=False)
    
df['Fecha'] = pd.to_datetime(df['Fecha'])

df = df.reset_index(drop=True)

map_columnas = {'Fecha':'date','Producto':'product',
                'Symbol/ISIN':'ISIN','Cantidad':'shares','Precio de ':'price',
                'Valor local':'amount_to_drop','Valor en EUR':'amount'}

df = df.rename(columns = map_columnas)
df = df.drop(columns='amount_to_drop')

### Check for cash position

In [ ]:
mask_has_isin = df['ISIN'].notna()

df.loc[mask_has_isin, 'type'] = 'long'
df.loc[~mask_has_isin, 'type'] = 'cash'

#### Compute shares, with cash price equal to one.

In [ ]:
df['price'] = df.loc[:, 'price'].fillna(1.0)

df['shares'] = df['amount'] / df['price']

In [ ]:
positions_raw_df = df.sort_values('date')

positions_long_df = positions_raw_df.set_index(keys = ['type', 'date']).loc['long']
positions_cash_df = positions_raw_df.set_index(keys = ['type', 'date']).loc['cash']

positions_raw_df = positions_raw_df.set_index('date')

In [ ]:
assets_value_ss = positions_raw_df.reset_index().groupby('date')['amount'].apply(np.sum)

In [ ]:
returns = assets_value_ss.pct_change()

In [ ]:
def from_positions_to_total_return(input_df, copy = True):
    """
    Compute from positions DataFrame the portfolio's total return.
    
    Parameters
    ----------
    input_df: pd.DataFrame
    
    
    Notes
    -----
    Includes cash as a position.
    """
    if copy == True:
        df = input_df.copy()
    else:
        df = input_df
        
    # Group by date all the assets
    assets_value_ss = input_df.reset_index()
    assets_value_ss = assets_value_ss.groupby('date')
    
    # Sum up the values in the 'amount' column
    assets_value_ss = assets_value_ss['amount'].apply(np.sum)
    
    # Cumulative returns
    total_return_ss = assets_value_ss.pct_change().fillna(0.0).add(1.0).cumprod().sub(1.0)
    
    return total_return_ss

In [ ]:
assets_value_ss.plot()

In [ ]:
from_positions_to_total_return(positions_raw_df).plot()

Calculamos el NAV diario:

In [ ]:
nav_daily = df.groupby('Fecha')['Valor en EUR'].sum()

In [ ]:
nav_daily.plot()

Descargamos todos los estado de cuenta (sobre todo nos interesan los ingresos y retiradas):

In [ ]:
datetime_ini = datetime.strptime(date_ini, '%Y%m%d')
datetime_fin = datetime.strptime(date_fin, '%Y%m%d')

url_account_formated = url_account.format(int_account=intAccount, 
                                          session_id=sessionId, 
                                          day_i=datetime_ini.strftime('%d'), 
                                          month_i=datetime_ini.strftime('%m'), 
                                          year_i=datetime_ini.strftime('%Y'), 
                                          day_f=datetime_fin.strftime('%d'), 
                                          month_f=datetime_fin.strftime('%m'), 
                                          year_f=datetime_fin.strftime('%Y'))

urllib.request.urlretrieve(url_account_formated, path / 'Account.xls')

In [ ]:
df_account = pd.read_excel(path / 'Account.xls')
df_account['Fecha'] = pd.to_datetime(df_account['Fecha'])
df_account = df_account.rename(columns={'Variación':'curr_d', 'Unnamed: 8':'Dinero', 'Saldo':'curr_s', 'Unnamed: 10':'Saldo'})

df_movs_cash = df_account.loc[df_account['Descripción'].isin(['Ingreso', 'Retirada']),]
df_movs_cash

In [ ]:
from pandas.tseries.offsets import BDay

# Descontamos de los nav los ingresos, y le sumamos las retiradas
# Menos el primer ingreso de todos (ultima fila del df_mov_cash) que lo tomaremos como capital inicial.
flows = df_movs_cash.iloc[:-1, :]

flows_cumulative = pd.Series(0, index=nav_daily.index)
flows_cumulative.loc[flows['Fecha'] + BDay(0)] = flows['Dinero'].values * -1
flows_cumulative = flows_cumulative.cumsum()

nav_daily_con_flows = nav_daily + flows_cumulative

# pd.concat([nav_daily, flows_cumulative, nav_daily_con_flows], axis=1)

Generamos un reporte completo sobre la curva NAV de nuestra cuenta:

In [ ]:
nav_daily_period = nav_daily_con_flows  #.loc[:'2019-11-18']

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
### benchmark can be equal to any ticker of yahoo finance (https://finance.yahoo.com/quote/__ticker__/)  ###

# qs.reports.metrics(nav_daily_period, mode='basic', benchmark='^IBEX')  # shows basic/full metrics
# qs.reports.plots(nav_daily_period, mode='full', benchmark='SPY')  # shows basic/full plots
# qs.reports.basic(nav_daily_period, benchmark=None)  # shows basic metrics and plots
qs.reports.full(nav_daily_period, benchmark='^IBEX')  # shows full metrics and plots
# qs.reports.html(nav_daily_period, benchmark='^IBEX', output=path/'full_report.html')  # generates a complete report as html